In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps

2024-01-05 15:57:52.181827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 15:57:52.827373: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
userknncf_study = optuna.create_study(
    study_name="UserKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3beta_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-05 15:58:07,450] Using an existing study with name 'ItemKNNCF' instead of creating a new one.
[I 2024-01-05 15:58:23,798] Using an existing study with name 'UserKNNCF' instead of creating a new one.
[I 2024-01-05 15:58:39,797] Using an existing study with name 'P3Beta' instead of creating a new one.


In [4]:
recommenders = [
    {
        "recommender": ItemKNNCFRecommender,
        "params": itemknncf_study.best_params,
    },
    {
        "recommender": UserKNNCFRecommender,
        "params": userknncf_study.best_params,
    },
    {
        "recommender": RP3betaRecommender,
        "params": p3beta_study.best_params,
    }
]

In [8]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, data_train, recommenders):
        super(ScoresHybridRecommender, self).__init__(data_train)

        self.data_train = sps.csr_matrix(data_train)
        self.recommenders = recommenders
        
    def prefit(self):
        for rec in self.recommenders:
            rec["recommender"] = rec["recommender"](self.data_train)
            rec["recommender"].fit(**rec["params"])
        
        
    def fit(self, weights):
        self.weights = weights      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights = []
        for rec in self.recommenders:
            item_weights.append(rec["recommender"]._compute_item_score(user_id_array, items_to_compute))

        item_weights = sum([a*b for a,b in zip(item_weights, self.weights)])
        return item_weights

In [6]:
scores_hybrid_study = optuna.create_study(
    study_name="ScoresHybrid",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-05 15:59:06,640] Using an existing study with name 'ScoresHybrid' instead of creating a new one.


In [9]:
scores_hybrid_recommender = ScoresHybridRecommender(data_train, recommenders)
scores_hybrid_recommender.prefit()

def objective(trial):
    weights = []
    for i in range(len(recommenders)):
        weights.append(trial.suggest_uniform("weight_{}".format(i), 0, 1))
    scores_hybrid_recommender.fit(weights)
    _, _, ev_map, _, _ = evaluator(scores_hybrid_recommender, data_train, data_val)
    return ev_map

scores_hybrid_study.optimize(objective, n_trials=100)

ScoresHybridRecommender: URM Detected 215 ( 1.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 127 ( 0.6%) items with no interactions.
ItemKNNCFRecommender: URM Detected 215 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 127 ( 0.6%) items with no interactions.
Similarity column 22222 (100.0%), 9876.86 column/sec. Elapsed time 2.25 sec
UserKNNCFRecommender: URM Detected 215 ( 1.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 127 ( 0.6%) items with no interactions.
Similarity column 12638 (100.0%), 10062.87 column/sec. Elapsed time 1.26 sec
RP3betaRecommender: URM Detected 215 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 127 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4389.92 column/sec. Elapsed time 5.06 sec


/tmp/ipykernel_48274/2146444983.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weights.append(trial.suggest_uniform("weight_{}".format(i), 0, 1))
[I 2024-01-05 16:00:54,366] Trial 0 finished with value: 0.0748597494067432 and parameters: {'weight_0': 0.6939324663886626, 'weight_1': 0.22386061566316262, 'weight_2': 0.16925307885447627}. Best is trial 0 with value: 0.0748597494067432.
[I 2024-01-05 16:01:27,137] Trial 1 finished with value: 0.07336114968865434 and parameters: {'weight_0': 0.6522523581713598, 'weight_1': 0.47760906929164404, 'weight_2': 0.6541540365328569}. Best is trial 0 with value: 0.0748597494067432.
[I 2024-01-05 16:02:01,942] Trial 2 finished with value: 0.06608064622209452 and parameters: {'weight_0': 0.06789674663545808, 'weight_1': 0.8684945149522487, 'weight_2': 0.9323095869048201}. Best is trial 0 with value: 0.0748